In [ ]:
# @title 🚀 1. LIMPEZA TOTAL E INSTALAÇÃO
import os, subprocess, time, random

print("🧹 Removendo rastros de sessões anteriores...")
!pkill -9 -f engine.py
!pkill -9 cloudflared
!rm -f engine.py*

print("📦 Instalando dependências...")
!pip install -q diffusers transformers accelerate flask flask-cors

print("📥 Baixando engine.py direto do GitHub (Sem Cache)...")
# O sufixo v= aleatório força o download da versão mais recente
v = random.randint(1, 99999)
!wget -q "https://raw.githubusercontent.com/Miraplay2025/Gerador-de-imagem/main/engine.py?v={v}" -O engine.py

with open('engine.py', 'r') as f:
    content = f.read()
    if "@app.route('/')" in content:
        print("✅ SUCESSO: A versão nova com a rota '/' foi detectada!")
    else:
        print("❌ ERRO: O arquivo baixado ainda é a versão antiga. Verifique seu GitHub!")

In [ ]:
# @title 🔗 2. INICIAR MOTOR E GERAR LINK
import subprocess, re, time

print("⚙️ Iniciando servidor...")
subprocess.Popen(["python3", "engine.py"])
time.sleep(12) # Tempo extra para garantir que o Flask subiu

print("🌐 Criando túnel...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb > /dev/null 2>&1

p = subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://127.0.0.1:5000'], 
                     stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in p.stdout:
    if ".trycloudflare.com" in line:
        url = re.search(r"https://[a-zA-Z0-9-]+\.trycloudflare\.com", line)
        if url:
            print("\n" + "="*60)
            print(f"✅ LINK ATIVO: {url.group(0)}")
            print("="*60)
            break